In [1]:
#import statemets

from customScripts import utilities as util
from customScripts import features as feat

In [2]:
# parameters

#number of samples / second
sampling_rate = 48000

#length of frame in samples
frame_length = 48000

#number of samples used as offset for earch consecutive frame
hop_length = 24000

#number of mel frequency bins to use
bin_number = 40 # also try 80

#set the desired number of frames / second here
ground_thruth_conversion_const = 2

#input data
file_path = 'audio_files/Muppets-02-01-01.wav'
grount_truth_path = 'ground_truth/Muppets-02-01-01.csv'
file_length = 1547 #in seconds

In [17]:
#load audio files

y = util.load_audio(file_path, sampling_rate)

In [18]:
#compute spectrogram

spectrogram = feat.compute_spectrogram(y, frame_length, hop_length, bin_number)

In [38]:
#load ground truth, pad it with zeroes and adjust to frame number

gt_1 = util.load_ground_truth(grount_truth_path, '/n')
gt_1 = util.compute_0_padded_gt(gt_1, file_length)
gt_1 = util.adjust_gt_to_frames(gt_1, ground_thruth_conversion_const)
gt_1.append(0) # append a zero so that lenght equals the frames list length

#for i,g in enumerate(gt_1):
#    if g == 1:
#        print(i)